### Check GPU availability

In [1]:
!nvidia-smi

Thu Jan 17 01:55:39 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.145                Driver Version: 384.145                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 23%   30C    P8    17W / 250W |    760MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:03:00.0 Off |                  N/A |
| 23%   33C    P8    17W / 250W |    631MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

### Import libraries

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import math
import numpy as np
import time
import json
import plotly
import logging
logging.getLogger().setLevel(logging.INFO)

from pprint import pprint
from tqdm import tqdm_notebook
from idst_util import trivial
from idst_util import dstc2

from plotly.graph_objs import Scatter, Layout
from plotly.graph_objs.layout import Margin
plotly.offline.init_notebook_mode(connected = True)

[nltk_data] Downloading package punkt to /home/is/andrei-
[nltk_data]     cc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Check DSTC2 availability

In [3]:
trivial.print_idst()
dstc2.check()

INFO:root:+--------------------------------+
INFO:root:|         _ ____  ___________    |
INFO:root:|        (_) __ \/ ___/_  __/    |
INFO:root:|       / / / / /\__ \ / /       |
INFO:root:|      / / /_/ /___/ // /        |
INFO:root:|     /_/_____//____//_/         |
INFO:root:|                                |
INFO:root:+--------------------------------+
INFO:root:|Incremental Dialog State Tracker|
INFO:root:+--------------------------------+
INFO:root:+--------------------------------+
INFO:root:|     Dialog State Tracker 2     |
INFO:root:|         Data Checker           |
INFO:root:+--------------------------------+
INFO:root:Looking for dstc2 directory in .
INFO:root:dstc2 was found!
INFO:root:Looking for dstc2_traindev directory in ./dstc2
INFO:root:dstc2_traindev was found!
INFO:root:Looking for dstc2_test directory in ./dstc2
INFO:root:dstc2_test was found!
INFO:root:Looking for dstc2_scripts directory in ./dstc2
INFO:root:dstc2_scripts was found!
INFO:root:Done!


### Retrieve data

In [4]:
raw_X_train, raw_Y_train, \
raw_X_dev, raw_Y_dev, \
raw_X_test, raw_Y_test, \
ontology = dstc2.retrieve_raw_datasets(train_data_augmentation = True)

INFO:root:+--------------------------------+
INFO:root:|     Dialog State Tracker 2     |
INFO:root:|       Dataset Retrieval        |
INFO:root:+--------------------------------+
INFO:root:Reading dstc2_train.flist, dstc2_dev.flist and ontology_dstc2.json
INFO:root:Asserted 1612 dialogs for dstc2_train.flist
INFO:root:Asserted 506 dialogs for dstc2_dev.flist
INFO:root:Extracting raw train features
100%|██████████| 1612/1612 [00:10<00:00, 147.87it/s]
INFO:root:Extracting raw dev features
100%|██████████| 506/506 [00:03<00:00, 167.78it/s]
INFO:root:Reading dstc2_test.flist
INFO:root:Asserted 1117 dialogs for dstc2_test.flist
INFO:root:Extracting raw test features
100%|██████████| 1117/1117 [00:07<00:00, 145.78it/s]


### Set device

In [5]:
logging.info("+--------------------------------+")
logging.info("|            Baseline            |")
logging.info("+--------------------------------+")

GPU_ID = 0
DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")
#DEVICE = "cpu"
if DEVICE == "cpu":
    logging.warning("Running on CPU")
else:
    logging.info("Running on GPU {}".format(GPU_ID))

INFO:root:+--------------------------------+
INFO:root:|            Baseline            |
INFO:root:+--------------------------------+
INFO:root:Running on GPU 0


### Create vocabularies

In [6]:
logging.info("+--------------------------------+")
logging.info("|          Vocabulary            |")
logging.info("+--------------------------------+")
logging.info("Creating token_to_index, index_to_token and token_to_count dictionaries")

token_to_index = {"<unk>": 0}
index_to_token = {0: "<unk>"}
token_to_count = {"<unk>": 1}

for raw_train_dialog in tqdm_notebook(raw_X_train):
    
    for raw_train_turn in raw_train_dialog["turns"]:
        
        tokens_scores = raw_train_turn["system"] + raw_train_turn["user"]
        
        for token_score in tokens_scores:
            token = token_score[0]
            if token not in token_to_index:
                token_to_index[token] = len(token_to_index)
                index_to_token[len(token_to_index)] = token
                token_to_count[token] = 1
            else:
                token_to_count[token] += 1
                
assert len(token_to_index) == len(index_to_token)
assert len(token_to_index) == len(token_to_count)

INFO:root:+--------------------------------+
INFO:root:|          Vocabulary            |
INFO:root:+--------------------------------+
INFO:root:Creating token_to_index, index_to_token and token_to_count dictionaries


### Execution configuration

In [7]:
logging.info("+--------------------------------+")
logging.info("|         Configuration          |")
logging.info("+--------------------------------+")

VOCABULARY_SIZE = len(token_to_index)
logging.info("VOCABULARY_SIZE:\t\t{}".format(VOCABULARY_SIZE))

# NOTE: +2 because of null and dontcare
GOAL_FOOD_DIM = len(ontology["informable"]["food"]) + 2 
GOAL_PRICERANGE_DIM = len(ontology["informable"]["pricerange"]) + 2
GOAL_NAME_DIM = len(ontology["informable"]["name"]) + 2
GOAL_AREA_DIM = len(ontology["informable"]["area"]) + 2
METHOD_DIM = len(ontology["method"])
REQUESTED_DIM = len(ontology["requestable"])
EMBEDDING_DIM = 170
ALTERED_EMBEDDING_DIM = 300
HIDDEN_DIM = 100
NUM_EPOCHS = 30
GOAL_LOSS_FUNCTION = nn.CrossEntropyLoss()
METHOD_LOSS_FUNCTION = nn.CrossEntropyLoss()
REQUESTED_LOSS_FUNCTION = nn.BCELoss()
logging.info("GOAL_FOOD_DIM:\t\t{}".format(GOAL_FOOD_DIM))
logging.info("GOAL_PRICERANGE_DIM:\t\t{}".format(GOAL_PRICERANGE_DIM))
logging.info("GOAL_NAME_DIM:\t\t{}".format(GOAL_NAME_DIM))
logging.info("GOAL_AREA_DIM:\t\t{}".format(GOAL_AREA_DIM))
logging.info("METHOD_DIM:\t\t\t{}".format(METHOD_DIM))
logging.info("REQUESTED_DIM:\t\t{}".format(REQUESTED_DIM))
logging.info("EMBEDDING_DIM:\t\t{}".format(EMBEDDING_DIM))
logging.info("ALTERED_EMBEDDING_DIM:\t{}".format(ALTERED_EMBEDDING_DIM))
logging.info("HIDDEN_DIM:\t\t\t{}".format(HIDDEN_DIM))
logging.info("NUM_EPOCHS:\t\t\t{}".format(NUM_EPOCHS))
logging.info("GOAL_LOSS_FUNCTION:\t\t{}".format(GOAL_LOSS_FUNCTION))
logging.info("METHOD_LOSS_FUNCTION:\t\t{}".format(METHOD_LOSS_FUNCTION))
logging.info("REQUESTED_LOSS_FUNCTION:\t{}".format(REQUESTED_LOSS_FUNCTION))

INFO:root:+--------------------------------+
INFO:root:|         Configuration          |
INFO:root:+--------------------------------+
INFO:root:VOCABULARY_SIZE:		1149
INFO:root:GOAL_FOOD_DIM:		93
INFO:root:GOAL_PRICERANGE_DIM:		5
INFO:root:GOAL_NAME_DIM:		115
INFO:root:GOAL_AREA_DIM:		7
INFO:root:METHOD_DIM:			5
INFO:root:REQUESTED_DIM:		8
INFO:root:EMBEDDING_DIM:		170
INFO:root:ALTERED_EMBEDDING_DIM:	300
INFO:root:HIDDEN_DIM:			100
INFO:root:NUM_EPOCHS:			30
INFO:root:GOAL_LOSS_FUNCTION:		CrossEntropyLoss()
INFO:root:METHOD_LOSS_FUNCTION:		CrossEntropyLoss()
INFO:root:REQUESTED_LOSS_FUNCTION:	BCELoss()


### Models utilities

In [8]:
def get_index_and_score(turn, token_to_index, mode, device):
    indices = []
    scores = []
    if mode == "train":
        for system_token, system_token_score in turn["system"]:
            indices.append(token_to_index[system_token])
            scores.append(system_token_score)
        for user_token, user_token_score in turn["user"]:
            if np.random.binomial(n = 1, p = 0.1) == 1:
                indices.append(token_to_index["<unk>"])
            else:
                indices.append(token_to_index[user_token])
            scores.append(user_token_score)
    else:
        tokens_scores = turn["system"] + turn["user"]
        for token, score in tokens_scores:
            if token not in token_to_index:
                indices.append(token_to_index["<unk>"])
            else:
                indices.append(token_to_index[token])
            scores.append(score)
    assert len(indices) == len(scores)
    return torch.tensor(indices, dtype = torch.long, device = device), torch.tensor(scores, dtype = torch.float, device = device)

def plotly_plot(dev_accuracies, test_accuracies, accuracy_type):
    plotly.offline.iplot({
                            "data": [Scatter(
                                            x = list(range(len(dev_accuracies))),
                                            y = dev_accuracies,
                                            mode = "lines+markers",
                                            name = "Dev {} Accuracy".format(accuracy_type),
                                            marker = dict(color = "#3498db")),
                                    Scatter(
                                            x = list(range(len(test_accuracies))),
                                            y = test_accuracies,
                                            mode = "lines+markers",
                                            name = "Test {} Accuracy".format(accuracy_type),
                                            marker = dict(color = "#9b59b6"))],
                            "layout": Layout(
                                             title = "<b>Dev-Test {} Accuracy</b>".format(accuracy_type),
                                             xaxis = dict(title = "<b>Epoch</b>",
                                                          dtick = 1,
                                                          titlefont = dict(color = "#34495e")),
                                             yaxis = dict(title = "<b>Accuracy</b>",
                                                          titlefont = dict(color = "#34495e")),
                                             margin = Margin(b = 150))
                        })

class EarlyStopping():
    
    def __init__(self, min_delta = 0, patience = 1):
        
        self.min_delta = min_delta
        self.patience = patience
        self.wait = 0
        self.stopped_epoch = 0
        self.best = -np.Inf
        self.stop_training = False
    
    def on_epoch_end(self, epoch, current_value):
        if np.greater((current_value - self.min_delta), self.best):
            self.best = current_value
            self.wait = 0
        else:
            self.wait += 1
            if self.wait > self.patience:
                self.stopped_epoch = epoch
                self.stop_training = True
        return self.stop_training

get_bin = lambda x, n: format(x, "b").zfill(n)

def make_tracker(model_GoalPricerange, model_GoalArea, model_GoalName, model_GoalFood, model_Requested, model_Method, raw_X, raw_Y, dataset):
    
    model_GoalPricerange = model_GoalPricerange.eval()
    model_GoalArea = model_GoalArea.eval()
    model_GoalName = model_GoalName.eval()
    model_GoalFood = model_GoalFood.eval()
    model_Requested = model_Requested.eval()
    model_Method = model_Method.eval()
    
    with torch.no_grad():
        tracker_json = {}
        tracker_json["dataset"] = dataset
        tracker_json["sessions"] = []

        start_time = time.time()
        for raw_X_dialog, raw_Y_dialog in tqdm_notebook(zip(raw_X, raw_Y), total = len(raw_X)):
            
            model_GoalPricerange.hidden = model_GoalPricerange.init_hidden()
            model_GoalArea.hidden = model_GoalArea.init_hidden()
            model_GoalName.hidden = model_GoalName.init_hidden()
            model_GoalFood.hidden = model_GoalFood.init_hidden()
            model_Requested.hidden = model_Requested.init_hidden()
            model_Method.hidden = model_Method.init_hidden()
            
            session = {}
            session["session-id"] = raw_X_dialog["session-id"]
            session["turns"] = []

            for raw_X_turn, raw_Y_turn in zip(raw_X_dialog["turns"], raw_Y_dialog["turns"]):
                turn = {}
                turn["goal-labels"] = {}

                indices, scores = get_index_and_score(raw_X_turn, token_to_index, mode = "eval", device = DEVICE)
                
                goal_foods = model_GoalFood(indices, scores)
                goal_food = goal_foods[-1]
                
                goal_priceranges = model_GoalPricerange(indices, scores)
                goal_pricerange = goal_priceranges[-1]
                
                goal_names = model_GoalName(indices, scores)
                goal_name = goal_names[-1]
                
                goal_areas = model_GoalArea(indices, scores)
                goal_area = goal_areas[-1]
                
                requesteds = model_Requested(indices, scores)
                requested = requesteds[-1]
                
                methods = model_Method(indices, scores)
                method = methods[-1]
                
                turn["goal-labels"]["food"] = retrieve_output_GoalFood(goal_food, ontology)
                turn["goal-labels"]["pricerange"] = retrieve_output_GoalPricerange(goal_pricerange, ontology)
                turn["goal-labels"]["name"] = retrieve_output_GoalName(goal_name, ontology)
                turn["goal-labels"]["area"] = retrieve_output_GoalArea(goal_area, ontology)
                turn["requested-slots"] = retrieve_output_Requested(requested, ontology)
                turn["method-label"] = retrieve_output_Method(method, ontology)
                
                session["turns"].append(turn)
            tracker_json["sessions"].append(session)
        end_time = time.time()
        tracker_json["wall-time"] = end_time - start_time
        
        return tracker_json

def get_scores(model_GoalPricerange, model_GoalArea, model_GoalName, model_GoalFood, model_Requested, model_Method, raw_X, raw_Y, dataset):
    goal_accuracy = 0
    goal_l2 = 0
    requested_accuracy = 0
    requested_l2 = 0
    method_accuracy = 0
    method_l2 = 0
    
    tracker = make_tracker(model_GoalPricerange, model_GoalArea, model_GoalName, model_GoalFood, model_Requested, model_Method, raw_X, raw_Y, dataset = dataset)
    
    with open("tracker_panfmr_incremental.json", "w") as tracker_file:
        json.dump(tracker, tracker_file)
    
    if dataset == "dstc2_train":
        !python2 dstc2/dstc2_scripts/score.py\
        --dataset dstc2_train\
        --dataroot dstc2/dstc2_traindev/data\
        --ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
        --trackfile tracker_panfmr_incremental.json\
        --scorefile tracker_panfmr_incremental.score.csv
    elif dataset == "dstc2_dev":
        !python2 dstc2/dstc2_scripts/score.py\
        --dataset dstc2_dev\
        --dataroot dstc2/dstc2_traindev/data\
        --ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
        --trackfile tracker_panfmr_incremental.json\
        --scorefile tracker_panfmr_incremental.score.csv
    else:
        !python2 dstc2/dstc2_scripts/score.py\
        --dataset dstc2_test\
        --dataroot dstc2/dstc2_test/data\
        --ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
        --trackfile tracker_panfmr_incremental.json\
        --scorefile tracker_panfmr_incremental.score.csv

    file_cat = !python2 dstc2/dstc2_scripts/report.py --scorefile tracker_panfmr_incremental.score.csv
    
    found_accuracies = False
    for line in file_cat:
        if line.startswith("Accuracy"):
            accuracies = line.split("|")
            goal_accuracy = float(accuracies[1])
            requested_accuracy = float(accuracies[2])
            method_accuracy = float(accuracies[3])
            found_accuracies = True
        if found_accuracies and line.startswith("l2"):
            l2s = line.split("|")
            goal_l2 = float(l2s[1])
            method_l2 = float(l2s[2])
            requested_l2 = float(l2s[3])
    
    return goal_accuracy, goal_l2, requested_accuracy, requested_l2, method_accuracy, method_l2

def retrieve_gold_GoalFood(raw_Y, ontology, device):
    ontology_informable_food = ontology["informable"]["food"]
    raw_goal_food = raw_Y["goal"]["food"]
    goal_food = 0
    if raw_goal_food != None:
        if raw_goal_food == "dontcare":
            goal_food = 1
        else:    
            goal_food = ontology_informable_food.index(raw_goal_food) + 2
    return torch.tensor([goal_food], dtype = torch.long, device = device)

def retrieve_output_GoalFood(output_tensor, ontology):
    ontology_informable_food = ontology["informable"]["food"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_food_dict = {}
    
    index = torch.argmax(output_tensor).item()
    goal_food_dict["dontcare"] = output_tensor[1].item() 
    for index in range(len(output_tensor) - 2):
        goal_food_dict[ontology_informable_food[index]] = output_tensor[index + 2].item()
    return goal_food_dict

def retrieve_gold_GoalPriceRange(raw_Y, ontology, device):
    ontology_informable_pricerange = ontology["informable"]["pricerange"]
    raw_goal_pricerange = raw_Y["goal"]["pricerange"]
    goal_pricerange = 0
    if raw_goal_pricerange != None:
        if raw_goal_pricerange == "dontcare":
            goal_pricerange = 1
        else:    
            goal_pricerange = ontology_informable_pricerange.index(raw_goal_pricerange) + 2
    return torch.tensor([goal_pricerange], dtype = torch.long, device = device)

def retrieve_output_GoalPricerange(output_tensor, ontology):
    ontology_informable_pricerange = ontology["informable"]["pricerange"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_pricerange_dict = {}
    
    index = torch.argmax(output_tensor).item()
    goal_pricerange_dict["dontcare"] = output_tensor[1].item()
    for index in range(len(output_tensor) - 2):     
        goal_pricerange_dict[ontology_informable_pricerange[index]] = output_tensor[index + 2].item()
    return goal_pricerange_dict

def retrieve_gold_GoalName(raw_Y, ontology, device):
    ontology_informable_name = ontology["informable"]["name"]
    raw_goal_name = raw_Y["goal"]["name"]
    goal_name = 0
    if raw_goal_name != None:
        if raw_goal_name == "dontcare":
            goal_name = 1
        else:    
            goal_name = ontology_informable_name.index(raw_goal_name) + 2
    return torch.tensor([goal_name], dtype = torch.long, device = device)

def retrieve_output_GoalName(output_tensor, ontology):
    ontology_informable_name = ontology["informable"]["name"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_name_dict = {}
    
    index = torch.argmax(output_tensor).item()
    goal_name_dict["dontcare"] = output_tensor[1].item()
    for index in range(len(output_tensor) - 2):
        goal_name_dict[ontology_informable_name[index]] = output_tensor[index + 2].item()
        
    return goal_name_dict

def retrieve_gold_GoalArea(raw_Y, ontology, device):
    ontology_informable_area = ontology["informable"]["area"]
    raw_goal_area = raw_Y["goal"]["area"]
    goal_area = 0
    if raw_goal_area != None:
        if raw_goal_area == "dontcare":
            goal_area = 1
        else:    
            goal_area = ontology_informable_area.index(raw_goal_area) + 2
    return torch.tensor([goal_area], dtype = torch.long, device = device)

def retrieve_output_GoalArea(output_tensor, ontology):
    ontology_informable_area = ontology["informable"]["area"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_area_dict = {}
    
    index = torch.argmax(output_tensor).item()
    goal_area_dict["dontcare"] = output_tensor[1].item()
    for index in range(len(output_tensor) - 2):
        goal_area_dict[ontology_informable_area[index]] = output_tensor[index + 2].item()
        
    return goal_area_dict

def retrieve_gold_Method(raw_Y, ontology, device):
    ontology_method = ontology["method"]
    raw_gold_method = raw_Y["method"]
    gold_method = ontology_method.index(raw_gold_method)
    return torch.tensor([gold_method], dtype = torch.long, device = device)

def retrieve_output_Method(output_tensor, ontology):
    ontology_method = ontology["method"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    method_dict = {}
    
    for index in range(len(output_tensor)):
        method_dict[ontology_method[index]] = output_tensor[index].item()
    
    return method_dict

def retrieve_gold_Requested(raw_Y, ontology, device):
    ontology_requestable = ontology["requestable"]
    raw_gold_requested = raw_Y["requested"]
    gold_requested = np.zeros(len(ontology_requestable), dtype = float)
    if len(raw_gold_requested) != 0:
        for requested in raw_gold_requested:
            gold_requested[ontology_requestable.index(requested)] = 1.0
    return torch.tensor([gold_requested], dtype = torch.float, device = device)

def retrieve_output_Requested(output_tensor, ontology):
    ontology_requestable = ontology["requestable"]
    output_tensor = output_tensor.view(-1)
    requested_dict = {}
    for index in range(len(output_tensor)):
        probability_value = output_tensor[index].item()
        if np.greater_equal(probability_value, 0.5):
            requested_dict[ontology_requestable[index]] = probability_value
    return requested_dict

### Goal Pricerange Model

In [9]:
class GoalPricerangeModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 goal_pricerange_dim,
                 device):
        
        super(GoalPricerangeModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.goal_pricerange_dim = goal_pricerange_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.goal_pricerange_classifier = nn.Linear(in_features = hidden_dim,
                                                    out_features = goal_pricerange_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        goal_pricerange = F.log_softmax(self.goal_pricerange_classifier(lstm_out).view(-1, self.goal_pricerange_dim), dim = 1) 
        
        return goal_pricerange

model_GoalPricerange = GoalPricerangeModel(vocabulary_size = VOCABULARY_SIZE,
                                           embedding_dim = EMBEDDING_DIM,
                                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                           hidden_dim = HIDDEN_DIM,
                                           goal_pricerange_dim = GOAL_PRICERANGE_DIM,
                                           device = DEVICE)

model_GoalPricerange = model_GoalPricerange.to(DEVICE)

optimizer_GoalPricerangeModel = optim.Adam(model_GoalPricerange.parameters(), lr = 1e-4)

### Goal Area Model

In [10]:
class GoalAreaModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 goal_area_dim,
                 device):
        
        super(GoalAreaModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.goal_area_dim = goal_area_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.goal_area_classifier = nn.Linear(in_features = hidden_dim,
                                              out_features = goal_area_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        goal_area = F.log_softmax(self.goal_area_classifier(lstm_out).view(-1, self.goal_area_dim), dim = 1)
        
        return goal_area

model_GoalArea = GoalAreaModel(vocabulary_size = VOCABULARY_SIZE,
                               embedding_dim = EMBEDDING_DIM,
                               altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                               hidden_dim = HIDDEN_DIM,
                               goal_area_dim = GOAL_AREA_DIM,
                               device = DEVICE)

model_GoalArea = model_GoalArea.to(DEVICE)

optimizer_GoalAreaModel = optim.Adam(model_GoalArea.parameters(), lr = 1e-4)

### Goal Name Model

In [11]:
class GoalNameModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 goal_name_dim,
                 device):
        
        super(GoalNameModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.goal_name_dim = goal_name_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.goal_name_classifier = nn.Linear(in_features = hidden_dim,
                                              out_features = goal_name_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        goal_name = F.log_softmax(self.goal_name_classifier(lstm_out).view(-1, self.goal_name_dim), dim = 1)
        
        return goal_name

model_GoalName = GoalNameModel(vocabulary_size = VOCABULARY_SIZE,
                               embedding_dim = EMBEDDING_DIM,
                               altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                               hidden_dim = HIDDEN_DIM,
                               goal_name_dim = GOAL_NAME_DIM,
                               device = DEVICE)

model_GoalName = model_GoalName.to(DEVICE)

optimizer_GoalNameModel = optim.Adam(model_GoalName.parameters(), lr = 1e-4)

### Goal Food Model

In [12]:
class GoalFoodModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 goal_food_dim,
                 device):
        
        super(GoalFoodModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.goal_food_dim = goal_food_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.goal_food_classifier = nn.Linear(in_features = hidden_dim,
                                              out_features = goal_food_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        goal_food = F.log_softmax(self.goal_food_classifier(lstm_out).view(-1, self.goal_food_dim), dim = 1)        
        
        return goal_food

model_GoalFood = GoalFoodModel(vocabulary_size = VOCABULARY_SIZE,
                               embedding_dim = EMBEDDING_DIM,
                               altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                               hidden_dim = HIDDEN_DIM,
                               goal_food_dim = GOAL_FOOD_DIM,
                               device = DEVICE)

model_GoalFood = model_GoalFood.to(DEVICE)

optimizer_GoalFoodModel = optim.Adam(model_GoalFood.parameters(), lr = 1e-4)

### Requested Model

In [13]:
class RequestedModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 requested_dim,
                 device):
        
        super(RequestedModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.requested_dim = requested_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.requested_classifier = nn.Linear(in_features = hidden_dim,
                                              out_features = requested_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        requested = torch.sigmoid(self.requested_classifier(lstm_out).view(-1, self.requested_dim))
        
        return requested

model_Requested = RequestedModel(vocabulary_size = VOCABULARY_SIZE,
                                 embedding_dim = EMBEDDING_DIM,
                                 altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                 hidden_dim = HIDDEN_DIM,
                                 requested_dim = REQUESTED_DIM,
                                 device = DEVICE)

model_Requested = model_Requested.to(DEVICE)

optimizer_RequestedModel = optim.Adam(model_Requested.parameters(), lr = 1e-4)

### Method Model

In [14]:
class MethodModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 method_dim,
                 device):
        
        super(MethodModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.method_dim = method_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.method_classifier = nn.Linear(in_features = hidden_dim,
                                           out_features = method_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        method = F.log_softmax(self.method_classifier(lstm_out).view(-1, self.method_dim), dim = 1)
        
        return method

model_Method = MethodModel(vocabulary_size = VOCABULARY_SIZE,
                           embedding_dim = EMBEDDING_DIM,
                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                           hidden_dim = HIDDEN_DIM,
                           method_dim = METHOD_DIM,
                           device = DEVICE)

model_Method = model_Method.to(DEVICE)

optimizer_MethodModel = optim.Adam(model_Method.parameters(), lr = 1e-4)

### Train Goal Pricerange Model

In [19]:
dev_goal_accuracies = []
dev_requested_accuracies = []
dev_method_accuracies = []

test_goal_accuracies = []
test_requested_accuracies = []
test_method_accuracies = []

goal_pricerange_early_stopping = EarlyStopping(patience = 2)

train_indices = np.arange(raw_X_train.shape[0])

for epoch in range(NUM_EPOCHS):
    
    np.random.shuffle(train_indices)
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_GoalPricerange = model_GoalPricerange.train()
    
    for raw_X_train_dialog, raw_Y_train_dialog in tqdm_notebook(zip(raw_X_train[train_indices], raw_Y_train[train_indices]), total = len(raw_X_train)):

        model_GoalPricerange.hidden = model_GoalPricerange.init_hidden()
                        
        for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):
            
            optimizer_GoalPricerangeModel.zero_grad()
            
            turn_loss = 0
            
            indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)

            goal_priceranges = model_GoalPricerange(indices, scores)

            for goal_pricerange in goal_priceranges:
                
                goal_pricerange = goal_pricerange.unsqueeze(0)
                
                loss_goal_pricerange = GOAL_LOSS_FUNCTION(goal_pricerange,
                                                          retrieve_gold_GoalPriceRange(raw_Y_train_turn,
                                                                                       ontology = ontology,
                                                                                       device = DEVICE))
                turn_loss += loss_goal_pricerange
                
            turn_loss.backward(retain_graph = True)

            optimizer_GoalPricerangeModel.step()

    dev_goal_accuracy, \
    dev_goal_l2, \
    dev_requested_accuracy, \
    dev_requested_l2, \
    dev_method_accuracy, \
    dev_method_l2 = get_scores(model_GoalPricerange,
                               model_GoalArea,
                               model_GoalName,
                               model_GoalFood,
                               model_Requested,
                               model_Method,
                               raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")
    logging.info("DEV Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_goal_accuracy,
                                                                 np.around(dev_goal_accuracy, decimals = 2),
                                                                 dev_requested_accuracy,
                                                                 np.around(dev_requested_accuracy, decimals = 2),
                                                                 dev_method_accuracy,
                                                                 np.around(dev_method_accuracy, decimals = 2)))
    logging.info("DEV L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_goal_l2,
                                                                np.around(dev_goal_l2, decimals = 2),
                                                                dev_requested_l2,
                                                                np.around(dev_requested_l2, decimals = 2),
                                                                dev_method_l2,
                                                                np.around(dev_method_l2, decimals = 2)))
    dev_goal_accuracies.append(dev_goal_accuracy)
    dev_requested_accuracies.append(dev_requested_accuracy)
    dev_method_accuracies.append(dev_method_accuracy)

    test_goal_accuracy, \
    test_goal_l2, \
    test_requested_accuracy, \
    test_requested_l2, \
    test_method_accuracy, \
    test_method_l2 = get_scores(model_GoalPricerange,
                                model_GoalArea,
                                model_GoalName,
                                model_GoalFood,
                                model_Requested,
                                model_Method,
                                raw_X_test, raw_Y_test, dataset = "dstc2_test")
    logging.info("TEST Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(test_goal_accuracy,
                                                                  np.around(test_goal_accuracy, decimals = 2),
                                                                  test_requested_accuracy,
                                                                  np.around(test_requested_accuracy, decimals = 2),
                                                                  test_method_accuracy,
                                                                  np.around(test_method_accuracy, decimals = 2)))
    logging.info("TEST L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(test_goal_l2,
                                                                 np.around(test_goal_l2, decimals = 2),
                                                                 test_requested_l2,
                                                                 np.around(test_requested_l2, decimals = 2),
                                                                 test_method_l2,
                                                                 np.around(test_method_l2, decimals = 2)))
    test_goal_accuracies.append(test_goal_accuracy)
    test_requested_accuracies.append(test_requested_accuracy)
    test_method_accuracies.append(test_requested_accuracy)
        
    goal_pricerange_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_goal_accuracy))
    
    if goal_pricerange_early_stopping.stop_training:
        break

plotly_plot(dev_goal_accuracies, test_goal_accuracies, "Goal")

INFO:root:Epoch	1/30


INFO:root:DEV Acc:		0.0148592(0.01)		0.4572665(0.46)		0.0206932(0.02)
INFO:root:DEV L2:		0.9910679(0.99)		0.81642(0.82)		0.6142292(0.61)


INFO:root:TEST Acc:		0.0287955(0.03)		0.4820281(0.48)		0.0166256(0.02)
INFO:root:TEST L2:		0.9921205(0.99)		0.8182356(0.82)		0.6121364(0.61)
INFO:root:Epoch	2/30


INFO:root:DEV Acc:		0.0156413(0.02)		0.4572665(0.46)		0.0206932(0.02)
INFO:root:DEV L2:		0.9908609(0.99)		0.81642(0.82)		0.6142292(0.61)


INFO:root:TEST Acc:		0.0303437(0.03)		0.4820281(0.48)		0.0166256(0.02)
INFO:root:TEST L2:		0.9909904(0.99)		0.8182356(0.82)		0.6121364(0.61)
INFO:root:Epoch	3/30


...

INFO:root:TEST Acc:		0.0301373(0.03)		0.4820281(0.48)		0.0166256(0.02)
INFO:root:TEST L2:		0.9915199(0.99)		0.8182356(0.82)		0.6121364(0.61)
INFO:root:Epoch	4/30


INFO:root:DEV Acc:		0.0151199(0.02)		0.4572665(0.46)		0.0206932(0.02)
INFO:root:DEV L2:		0.9912878(0.99)		0.81642(0.82)		0.6142292(0.61)


INFO:root:TEST Acc:		0.0302405(0.03)		0.4820281(0.48)		0.0166256(0.02)
INFO:root:TEST L2:		0.9914662(0.99)		0.8182356(0.82)		0.6121364(0.61)
INFO:root:Epoch	5/30


INFO:root:DEV Acc:		0.0153806(0.02)		0.4572665(0.46)		0.0206932(0.02)
INFO:root:DEV L2:		0.9911074(0.99)		0.81642(0.82)		0.6142292(0.61)


INFO:root:TEST Acc:		0.0300341(0.03)		0.4820281(0.48)		0.0166256(0.02)
INFO:root:TEST L2:		0.991368(0.99)		0.8182356(0.82)		0.6121364(0.61)


### Save Goal Pricerange Model

In [20]:
torch.save(model_GoalPricerange.state_dict(), "model_PANFMR_incremental_GoalPricerange.pt")

### Load Goal Pricerange Model

In [21]:
model_GoalPricerange = GoalPricerangeModel(vocabulary_size = VOCABULARY_SIZE,
                                           embedding_dim = EMBEDDING_DIM,
                                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                           hidden_dim = HIDDEN_DIM,
                                           goal_pricerange_dim = GOAL_PRICERANGE_DIM,
                                           device = DEVICE)

model_GoalPricerange = model_GoalPricerange.to(DEVICE)
model_GoalPricerange.load_state_dict(torch.load("model_PANFMR_incremental_GoalPricerange.pt"))
model_GoalPricerange.eval()

GoalPricerangeModel(
  (embeddings): Embedding(1149, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (goal_pricerange_classifier): Linear(in_features=100, out_features=5, bias=True)
)

In [22]:
get_scores(model_GoalPricerange,
           model_GoalArea,
           model_GoalName,
           model_GoalFood,
           model_Requested,
           model_Method,
           raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")

(0.0153806, 0.9911074, 0.4572665, 0.81642, 0.0206932, 0.6142292)

In [23]:
get_scores(model_GoalPricerange,
           model_GoalArea,
           model_GoalName,
           model_GoalFood,
           model_Requested,
           model_Method,
           raw_X_test, raw_Y_test, dataset = "dstc2_test")

(0.0300341, 0.991368, 0.4820281, 0.8182356, 0.0166256, 0.6121364)

### Train Goal Area Model

In [24]:
dev_goal_accuracies = []
dev_requested_accuracies = []
dev_method_accuracies = []

test_goal_accuracies = []
test_requested_accuracies = []
test_method_accuracies = []

goal_area_early_stopping = EarlyStopping(patience = 2)

train_indices = np.arange(raw_X_train.shape[0])

for epoch in range(NUM_EPOCHS):
    
    np.random.shuffle(train_indices)
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_GoalArea = model_GoalArea.train()
    
    for raw_X_train_dialog, raw_Y_train_dialog in tqdm_notebook(zip(raw_X_train[train_indices], raw_Y_train[train_indices]), total = len(raw_X_train)):

        model_GoalArea.hidden = model_GoalArea.init_hidden()

        for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):
            
            optimizer_GoalAreaModel.zero_grad()
        
            turn_loss = 0
            
            indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)

            goal_areas = model_GoalArea(indices, scores)
            
            for goal_area in goal_areas:

                goal_area = goal_area.unsqueeze(0)
                
                loss_goal_area = GOAL_LOSS_FUNCTION(goal_area,
                                                    retrieve_gold_GoalArea(raw_Y_train_turn,
                                                                           ontology = ontology,
                                                                           device = DEVICE))
                turn_loss += loss_goal_area
                
            turn_loss.backward(retain_graph = True)

            optimizer_GoalAreaModel.step()
    
    dev_goal_accuracy, \
    dev_goal_l2, \
    dev_requested_accuracy, \
    dev_requested_l2, \
    dev_method_accuracy, \
    dev_method_l2 = get_scores(model_GoalPricerange,
                               model_GoalArea,
                               model_GoalName,
                               model_GoalFood,
                               model_Requested,
                               model_Method,
                               raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")
    logging.info("DEV Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_goal_accuracy,
                                                                 np.around(dev_goal_accuracy, decimals = 2),
                                                                 dev_requested_accuracy,
                                                                 np.around(dev_requested_accuracy, decimals = 2),
                                                                 dev_method_accuracy,
                                                                 np.around(dev_method_accuracy, decimals = 2)))
    logging.info("DEV L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_goal_l2,
                                                                np.around(dev_goal_l2, decimals = 2),
                                                                dev_requested_l2,
                                                                np.around(dev_requested_l2, decimals = 2),
                                                                dev_method_l2,
                                                                np.around(dev_method_l2, decimals = 2)))
    dev_goal_accuracies.append(dev_goal_accuracy)
    dev_requested_accuracies.append(dev_requested_accuracy)
    dev_method_accuracies.append(dev_method_accuracy)

    test_goal_accuracy, \
    test_goal_l2, \
    test_requested_accuracy, \
    test_requested_l2, \
    test_method_accuracy, \
    test_method_l2 = get_scores(model_GoalPricerange,
                                model_GoalArea,
                                model_GoalName,
                                model_GoalFood,
                                model_Requested,
                                model_Method,
                                raw_X_test, raw_Y_test, dataset = "dstc2_test")
    logging.info("TEST Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(test_goal_accuracy,
                                                                  np.around(test_goal_accuracy, decimals = 2),
                                                                  test_requested_accuracy,
                                                                  np.around(test_requested_accuracy, decimals = 2),
                                                                  test_method_accuracy,
                                                                  np.around(test_method_accuracy, decimals = 2)))
    logging.info("TEST L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(test_goal_l2,
                                                                 np.around(test_goal_l2, decimals = 2),
                                                                 test_requested_l2,
                                                                 np.around(test_requested_l2, decimals = 2),
                                                                 test_method_l2,
                                                                 np.around(test_method_l2, decimals = 2)))
    test_goal_accuracies.append(test_goal_accuracy)
    test_requested_accuracies.append(test_requested_accuracy)
    test_method_accuracies.append(test_requested_accuracy)
        
    goal_area_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_goal_accuracy))
    
    if goal_area_early_stopping.stop_training:
        break

plotly_plot(dev_goal_accuracies, test_goal_accuracies, "Goal")

INFO:root:Epoch	1/30


INFO:root:DEV Acc:		0.1160063(0.12)		0.4572665(0.46)		0.0206932(0.02)
INFO:root:DEV L2:		0.9604749(0.96)		0.81642(0.82)		0.6142292(0.61)


INFO:root:TEST Acc:		0.1831974(0.18)		0.4820281(0.48)		0.0166256(0.02)
INFO:root:TEST L2:		0.958017(0.96)		0.8182356(0.82)		0.6121364(0.61)
INFO:root:Epoch	2/30


INFO:root:DEV Acc:		0.1165276(0.12)		0.4572665(0.46)		0.0206932(0.02)
INFO:root:DEV L2:		0.9592414(0.96)		0.81642(0.82)		0.6142292(0.61)


INFO:root:TEST Acc:		0.1852616(0.19)		0.4820281(0.48)		0.0166256(0.02)
INFO:root:TEST L2:		0.9587613(0.96)		0.8182356(0.82)		0.6121364(0.61)
INFO:root:Epoch	3/30


...

INFO:root:TEST Acc:		0.1884611(0.19)		0.4820281(0.48)		0.0166256(0.02)
INFO:root:TEST L2:		0.9556447(0.96)		0.8182356(0.82)		0.6121364(0.61)
INFO:root:Epoch	5/30


INFO:root:DEV Acc:		0.1167883(0.12)		0.4572665(0.46)		0.0206932(0.02)
INFO:root:DEV L2:		0.9592725(0.96)		0.81642(0.82)		0.6142292(0.61)


INFO:root:TEST Acc:		0.1847456(0.18)		0.4820281(0.48)		0.0166256(0.02)
INFO:root:TEST L2:		0.9590725(0.96)		0.8182356(0.82)		0.6121364(0.61)
INFO:root:Epoch	6/30


INFO:root:DEV Acc:		0.1173097(0.12)		0.4572665(0.46)		0.0206932(0.02)
INFO:root:DEV L2:		0.9605633(0.96)		0.81642(0.82)		0.6142292(0.61)


INFO:root:TEST Acc:		0.1839199(0.18)		0.4820281(0.48)		0.0166256(0.02)
INFO:root:TEST L2:		0.9589216(0.96)		0.8182356(0.82)		0.6121364(0.61)


### Save Goal Area Model

In [25]:
torch.save(model_GoalArea.state_dict(), "model_PANFRM_incremental_GoalArea.pt")

### Load Goal Area Model

In [26]:
model_GoalArea = GoalAreaModel(vocabulary_size = VOCABULARY_SIZE,
                               embedding_dim = EMBEDDING_DIM,
                               altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                               hidden_dim = HIDDEN_DIM,
                               goal_area_dim = GOAL_AREA_DIM,
                               device = DEVICE)

model_GoalArea = model_GoalArea.to(DEVICE)
model_GoalArea.load_state_dict(torch.load("model_PANFRM_incremental_GoalArea.pt"))
model_GoalArea.eval()

GoalAreaModel(
  (embeddings): Embedding(1149, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (goal_area_classifier): Linear(in_features=100, out_features=7, bias=True)
)

In [27]:
get_scores(model_GoalPricerange,
           model_GoalArea,
           model_GoalName,
           model_GoalFood,
           model_Requested,
           model_Method,
           raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")

(0.1173097, 0.9605633, 0.4572665, 0.81642, 0.0206932, 0.6142292)

In [28]:
get_scores(model_GoalPricerange,
           model_GoalArea,
           model_GoalName,
           model_GoalFood,
           model_Requested,
           model_Method,
           raw_X_test, raw_Y_test, dataset = "dstc2_test")

(0.1839199, 0.9589216, 0.4820281, 0.8182356, 0.0166256, 0.6121364)

### Train Goal Name Model

In [29]:
dev_goal_accuracies = []
dev_requested_accuracies = []
dev_method_accuracies = []

test_goal_accuracies = []
test_requested_accuracies = []
test_method_accuracies = []

goal_name_early_stopping = EarlyStopping(patience = 2)

train_indices = np.arange(raw_X_train.shape[0])

for epoch in range(NUM_EPOCHS):
    
    np.random.shuffle(train_indices)
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_GoalName = model_GoalName.train()
    
    for raw_X_train_dialog, raw_Y_train_dialog in tqdm_notebook(zip(raw_X_train[train_indices], raw_Y_train[train_indices]), total = len(raw_X_train)):

        model_GoalName.hidden = model_GoalName.init_hidden()

        for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):
            
            optimizer_GoalNameModel.zero_grad()
        
            turn_loss = 0

            indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)
            
            goal_names = model_GoalName(indices, scores)
            
            for goal_name in goal_names:
                
                goal_name = goal_name.unsqueeze(0)
                
                loss_goal_name = GOAL_LOSS_FUNCTION(goal_name,
                                                    retrieve_gold_GoalName(raw_Y_train_turn,
                                                                           ontology = ontology,
                                                                           device = DEVICE))
                turn_loss += loss_goal_name
                
            turn_loss.backward(retain_graph = True)

            optimizer_GoalNameModel.step()
    
    dev_goal_accuracy, \
    dev_goal_l2, \
    dev_requested_accuracy, \
    dev_requested_l2, \
    dev_method_accuracy, \
    dev_method_l2 = get_scores(model_GoalPricerange,
                               model_GoalArea,
                               model_GoalName,
                               model_GoalFood,
                               model_Requested,
                               model_Method,
                               raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")
    logging.info("DEV Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_goal_accuracy,
                                                                 np.around(dev_goal_accuracy, decimals = 2),
                                                                 dev_requested_accuracy,
                                                                 np.around(dev_requested_accuracy, decimals = 2),
                                                                 dev_method_accuracy,
                                                                 np.around(dev_method_accuracy, decimals = 2)))
    logging.info("DEV L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_goal_l2,
                                                                np.around(dev_goal_l2, decimals = 2),
                                                                dev_requested_l2,
                                                                np.around(dev_requested_l2, decimals = 2),
                                                                dev_method_l2,
                                                                np.around(dev_method_l2, decimals = 2)))
    dev_goal_accuracies.append(dev_goal_accuracy)
    dev_requested_accuracies.append(dev_requested_accuracy)
    dev_method_accuracies.append(dev_method_accuracy)

    test_goal_accuracy, \
    test_goal_l2, \
    test_requested_accuracy, \
    test_requested_l2, \
    test_method_accuracy, \
    test_method_l2 = get_scores(model_GoalPricerange,
                                model_GoalArea,
                                model_GoalName,
                                model_GoalFood,
                                model_Requested,
                                model_Method,
                                raw_X_test, raw_Y_test, dataset = "dstc2_test")
    logging.info("TEST Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(test_goal_accuracy,
                                                                  np.around(test_goal_accuracy, decimals = 2),
                                                                  test_requested_accuracy,
                                                                  np.around(test_requested_accuracy, decimals = 2),
                                                                  test_method_accuracy,
                                                                  np.around(test_method_accuracy, decimals = 2)))
    logging.info("TEST L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(test_goal_l2,
                                                                 np.around(test_goal_l2, decimals = 2),
                                                                 test_requested_l2,
                                                                 np.around(test_requested_l2, decimals = 2),
                                                                 test_method_l2,
                                                                 np.around(test_method_l2, decimals = 2)))
    test_goal_accuracies.append(test_goal_accuracy)
    test_requested_accuracies.append(test_requested_accuracy)
    test_method_accuracies.append(test_requested_accuracy)
        
    goal_name_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_goal_accuracy))
    
    if goal_name_early_stopping.stop_training:
        break

plotly_plot(dev_goal_accuracies, test_goal_accuracies, "Goal")

INFO:root:Epoch	1/30


INFO:root:DEV Acc:		0.1173097(0.12)		0.4572665(0.46)		0.0206932(0.02)
INFO:root:DEV L2:		0.9638609(0.96)		0.81642(0.82)		0.6142292(0.61)


INFO:root:TEST Acc:		0.1839199(0.18)		0.4820281(0.48)		0.0166256(0.02)
INFO:root:TEST L2:		0.9619787(0.96)		0.8182356(0.82)		0.6121364(0.61)
INFO:root:Epoch	2/30


INFO:root:DEV Acc:		0.1173097(0.12)		0.4572665(0.46)		0.0206932(0.02)
INFO:root:DEV L2:		0.9636942(0.96)		0.81642(0.82)		0.6142292(0.61)


INFO:root:TEST Acc:		0.1839199(0.18)		0.4820281(0.48)		0.0166256(0.02)
INFO:root:TEST L2:		0.9618674(0.96)		0.8182356(0.82)		0.6121364(0.61)
INFO:root:Epoch	3/30


INFO:root:DEV Acc:		0.1173097(0.12)		0.4572665(0.46)		0.0206932(0.02)
INFO:root:DEV L2:		0.9638622(0.96)		0.81642(0.82)		0.6142292(0.61)


INFO:root:TEST Acc:		0.1839199(0.18)		0.4820281(0.48)		0.0166256(0.02)
INFO:root:TEST L2:		0.9619856(0.96)		0.8182356(0.82)		0.6121364(0.61)
INFO:root:Epoch	4/30


INFO:root:DEV Acc:		0.1173097(0.12)		0.4572665(0.46)		0.0206932(0.02)
INFO:root:DEV L2:		0.9637806(0.96)		0.81642(0.82)		0.6142292(0.61)


INFO:root:TEST Acc:		0.1839199(0.18)		0.4820281(0.48)		0.0166256(0.02)
INFO:root:TEST L2:		0.9619596(0.96)		0.8182356(0.82)		0.6121364(0.61)


### Save Goal Name Model

In [30]:
torch.save(model_GoalName.state_dict(), "model_PANFMR_incremental_GoalName.pt")

### Load Goal Name Model

In [31]:
model_GoalName = GoalNameModel(vocabulary_size = VOCABULARY_SIZE,
                               embedding_dim = EMBEDDING_DIM,
                               altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                               hidden_dim = HIDDEN_DIM,
                               goal_name_dim = GOAL_NAME_DIM,
                               device = DEVICE)

model_GoalName = model_GoalName.to(DEVICE)
model_GoalName.load_state_dict(torch.load("model_PANFMR_incremental_GoalName.pt"))
model_GoalName.eval()

GoalNameModel(
  (embeddings): Embedding(1149, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (goal_name_classifier): Linear(in_features=100, out_features=115, bias=True)
)

In [32]:
get_scores(model_GoalPricerange,
           model_GoalArea,
           model_GoalName,
           model_GoalFood,
           model_Requested,
           model_Method,
           raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")

(0.1173097, 0.9637806, 0.4572665, 0.81642, 0.0206932, 0.6142292)

In [33]:
get_scores(model_GoalPricerange,
           model_GoalArea,
           model_GoalName,
           model_GoalFood,
           model_Requested,
           model_Method,
           raw_X_test, raw_Y_test, dataset = "dstc2_test")

(0.1839199, 0.9619596, 0.4820281, 0.8182356, 0.0166256, 0.6121364)

### Train Goal Food Model

In [34]:
dev_goal_accuracies = []
dev_requested_accuracies = []
dev_method_accuracies = []

test_goal_accuracies = []
test_requested_accuracies = []
test_method_accuracies = []

goal_food_early_stopping = EarlyStopping(patience = 2)

train_indices = np.arange(raw_X_train.shape[0])

for epoch in range(NUM_EPOCHS):
    
    np.random.shuffle(train_indices)
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_GoalFood = model_GoalFood.train()
    
    for raw_X_train_dialog, raw_Y_train_dialog in tqdm_notebook(zip(raw_X_train[train_indices], raw_Y_train[train_indices]), total = len(raw_X_train)):

        model_GoalFood.hidden = model_GoalFood.init_hidden()

        for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):
                    
            optimizer_GoalFoodModel.zero_grad()
        
            turn_loss = 0
            
            indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)
            
            goal_foods = model_GoalFood(indices, scores)
            
            for goal_food in goal_foods:
                
                goal_food = goal_food.unsqueeze(0)
                
                loss_goal_food = GOAL_LOSS_FUNCTION(goal_food,
                                                    retrieve_gold_GoalFood(raw_Y_train_turn,
                                                                           ontology = ontology,
                                                                           device = DEVICE))
                turn_loss += loss_goal_food
                
            turn_loss.backward(retain_graph = True)

            optimizer_GoalFoodModel.step()
    
    dev_goal_accuracy, \
    dev_goal_l2, \
    dev_requested_accuracy, \
    dev_requested_l2, \
    dev_method_accuracy, \
    dev_method_l2 = get_scores(model_GoalPricerange,
                               model_GoalArea,
                               model_GoalName,
                               model_GoalFood,
                               model_Requested,
                               model_Method,
                               raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")
    logging.info("DEV Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_goal_accuracy,
                                                                 np.around(dev_goal_accuracy, decimals = 2),
                                                                 dev_requested_accuracy,
                                                                 np.around(dev_requested_accuracy, decimals = 2),
                                                                 dev_method_accuracy,
                                                                 np.around(dev_method_accuracy, decimals = 2)))
    logging.info("DEV L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_goal_l2,
                                                                np.around(dev_goal_l2, decimals = 2),
                                                                dev_requested_l2,
                                                                np.around(dev_requested_l2, decimals = 2),
                                                                dev_method_l2,
                                                                np.around(dev_method_l2, decimals = 2)))
    dev_goal_accuracies.append(dev_goal_accuracy)
    dev_requested_accuracies.append(dev_requested_accuracy)
    dev_method_accuracies.append(dev_method_accuracy)

    test_goal_accuracy, \
    test_goal_l2, \
    test_requested_accuracy, \
    test_requested_l2, \
    test_method_accuracy, \
    test_method_l2 = get_scores(model_GoalPricerange,
                                model_GoalArea,
                                model_GoalName,
                                model_GoalFood,
                                model_Requested,
                                model_Method,
                                raw_X_test, raw_Y_test, dataset = "dstc2_test")
    logging.info("TEST Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(test_goal_accuracy,
                                                                  np.around(test_goal_accuracy, decimals = 2),
                                                                  test_requested_accuracy,
                                                                  np.around(test_requested_accuracy, decimals = 2),
                                                                  test_method_accuracy,
                                                                  np.around(test_method_accuracy, decimals = 2)))
    logging.info("TEST L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(test_goal_l2,
                                                                 np.around(test_goal_l2, decimals = 2),
                                                                 test_requested_l2,
                                                                 np.around(test_requested_l2, decimals = 2),
                                                                 test_method_l2,
                                                                 np.around(test_method_l2, decimals = 2)))
    test_goal_accuracies.append(test_goal_accuracy)
    test_requested_accuracies.append(test_requested_accuracy)
    test_method_accuracies.append(test_requested_accuracy)
        
    goal_food_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_goal_accuracy))
    
    if goal_food_early_stopping.stop_training:
        break

plotly_plot(dev_goal_accuracies, test_goal_accuracies, "Goal")

INFO:root:Epoch	1/30


INFO:root:DEV Acc:		0.5594369(0.56)		0.4572665(0.46)		0.0206932(0.02)
INFO:root:DEV L2:		0.6179199(0.62)		0.81642(0.82)		0.6142292(0.61)


INFO:root:TEST Acc:		0.5828259(0.58)		0.4820281(0.48)		0.0166256(0.02)
INFO:root:TEST L2:		0.5922178(0.59)		0.8182356(0.82)		0.6121364(0.61)
INFO:root:Epoch	2/30


INFO:root:DEV Acc:		0.5988008(0.6)		0.4572665(0.46)		0.0206932(0.02)
INFO:root:DEV L2:		0.5583862(0.56)		0.81642(0.82)		0.6142292(0.61)


INFO:root:TEST Acc:		0.6285478(0.63)		0.4820281(0.48)		0.0166256(0.02)
INFO:root:TEST L2:		0.5203424(0.52)		0.8182356(0.82)		0.6121364(0.61)
INFO:root:Epoch	3/30


...

INFO:root:TEST Acc:		0.6346372(0.63)		0.4820281(0.48)		0.0166256(0.02)
INFO:root:TEST L2:		0.5503111(0.55)		0.8182356(0.82)		0.6121364(0.61)
INFO:root:Epoch	10/30


INFO:root:DEV Acc:		0.6193952(0.62)		0.4572665(0.46)		0.0206932(0.02)
INFO:root:DEV L2:		0.5609688(0.56)		0.81642(0.82)		0.6142292(0.61)


INFO:root:TEST Acc:		0.638972(0.64)		0.4820281(0.48)		0.0166256(0.02)
INFO:root:TEST L2:		0.5369122(0.54)		0.8182356(0.82)		0.6121364(0.61)
INFO:root:Epoch	11/30


INFO:root:DEV Acc:		0.6139208(0.61)		0.4572665(0.46)		0.0206932(0.02)
INFO:root:DEV L2:		0.5781036(0.58)		0.81642(0.82)		0.6142292(0.61)


INFO:root:TEST Acc:		0.6282382(0.63)		0.4820281(0.48)		0.0166256(0.02)
INFO:root:TEST L2:		0.5631499(0.56)		0.8182356(0.82)		0.6121364(0.61)


### Save Goal Food Model

In [35]:
torch.save(model_GoalFood.state_dict(), "model_PANFMR_incremental_GoalFood.pt")

### Load Goal Food Model

In [36]:
model_GoalFood = GoalFoodModel(vocabulary_size = VOCABULARY_SIZE,
                               embedding_dim = EMBEDDING_DIM,
                               altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                               hidden_dim = HIDDEN_DIM,
                               goal_food_dim = GOAL_FOOD_DIM,
                               device = DEVICE)

model_GoalFood = model_GoalFood.to(DEVICE)
model_GoalFood.load_state_dict(torch.load("model_PANFMR_incremental_GoalFood.pt"))
model_GoalFood.eval()

GoalFoodModel(
  (embeddings): Embedding(1149, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (goal_food_classifier): Linear(in_features=100, out_features=93, bias=True)
)

In [37]:
get_scores(model_GoalPricerange,
           model_GoalArea,
           model_GoalName,
           model_GoalFood,
           model_Requested,
           model_Method,
           raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")

(0.6139208, 0.5781036, 0.4572665, 0.81642, 0.0206932, 0.6142292)

In [38]:
get_scores(model_GoalPricerange,
           model_GoalArea,
           model_GoalName,
           model_GoalFood,
           model_Requested,
           model_Method,
           raw_X_test, raw_Y_test, dataset = "dstc2_test")

(0.6282382, 0.5631499, 0.4820281, 0.8182356, 0.0166256, 0.6121364)

### Train Requested Model

In [39]:
dev_goal_accuracies = []
dev_requested_accuracies = []
dev_method_accuracies = []

test_goal_accuracies = []
test_requested_accuracies = []
test_method_accuracies = []

requested_early_stopping = EarlyStopping(patience = 2)

train_indices = np.arange(raw_X_train.shape[0])

for epoch in range(NUM_EPOCHS):
    
    np.random.shuffle(train_indices)
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_Requested = model_Requested.train()
    
    for raw_X_train_dialog, raw_Y_train_dialog in tqdm_notebook(zip(raw_X_train[train_indices], raw_Y_train[train_indices]), total = len(raw_X_train)):

        model_Requested.hidden = model_Requested.init_hidden()

        for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):
            
            optimizer_RequestedModel.zero_grad()
        
            turn_loss = 0
            
            indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)
            
            requesteds = model_Requested(indices, scores)

            for requested in requesteds:
                
                requested = requested.unsqueeze(0)
                
                loss_requested = REQUESTED_LOSS_FUNCTION(requested,
                                                         retrieve_gold_Requested(raw_Y_train_turn,
                                                                                 ontology = ontology,
                                                                                 device = DEVICE))
                turn_loss += loss_requested
                
            turn_loss.backward(retain_graph = True)

            optimizer_RequestedModel.step()
    
    dev_goal_accuracy, \
    dev_goal_l2, \
    dev_requested_accuracy, \
    dev_requested_l2, \
    dev_method_accuracy, \
    dev_method_l2 = get_scores(model_GoalPricerange,
                               model_GoalArea,
                               model_GoalName,
                               model_GoalFood,
                               model_Requested,
                               model_Method,
                               raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")
    logging.info("DEV Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_goal_accuracy,
                                                                 np.around(dev_goal_accuracy, decimals = 2),
                                                                 dev_requested_accuracy,
                                                                 np.around(dev_requested_accuracy, decimals = 2),
                                                                 dev_method_accuracy,
                                                                 np.around(dev_method_accuracy, decimals = 2)))
    logging.info("DEV L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_goal_l2,
                                                                np.around(dev_goal_l2, decimals = 2),
                                                                dev_requested_l2,
                                                                np.around(dev_requested_l2, decimals = 2),
                                                                dev_method_l2,
                                                                np.around(dev_method_l2, decimals = 2)))
    dev_goal_accuracies.append(dev_goal_accuracy)
    dev_requested_accuracies.append(dev_requested_accuracy)
    dev_method_accuracies.append(dev_method_accuracy)

    test_goal_accuracy, \
    test_goal_l2, \
    test_requested_accuracy, \
    test_requested_l2, \
    test_method_accuracy, \
    test_method_l2 = get_scores(model_GoalPricerange,
                                model_GoalArea,
                                model_GoalName,
                                model_GoalFood,
                                model_Requested,
                                model_Method,
                                raw_X_test, raw_Y_test, dataset = "dstc2_test")
    logging.info("TEST Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(test_goal_accuracy,
                                                              np.around(test_goal_accuracy, decimals = 2),
                                                              test_requested_accuracy,
                                                              np.around(test_requested_accuracy, decimals = 2),
                                                              test_method_accuracy,
                                                              np.around(test_method_accuracy, decimals = 2)))
    logging.info("TEST L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(test_goal_l2,
                                                              np.around(test_goal_l2, decimals = 2),
                                                              test_requested_l2,
                                                              np.around(test_requested_l2, decimals = 2),
                                                              test_method_l2,
                                                              np.around(test_method_l2, decimals = 2)))
    test_goal_accuracies.append(test_goal_accuracy)
    test_requested_accuracies.append(test_requested_accuracy)
    test_method_accuracies.append(test_requested_accuracy)
        
    requested_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_requested_accuracy))
    
    if requested_early_stopping.stop_training:
        break

plotly_plot(dev_requested_accuracies, test_requested_accuracies, "Requested")

INFO:root:Epoch	1/30


INFO:root:DEV Acc:		0.6139208(0.61)		0.8943383(0.89)		0.0206932(0.02)
INFO:root:DEV L2:		0.5781036(0.58)		0.81642(0.82)		0.2394439(0.24)


INFO:root:TEST Acc:		0.6282382(0.63)		0.9062337(0.91)		0.0166256(0.02)
INFO:root:TEST L2:		0.5631499(0.56)		0.8182356(0.82)		0.2171742(0.22)
INFO:root:Epoch	2/30


INFO:root:DEV Acc:		0.6139208(0.61)		0.9448251(0.94)		0.0206932(0.02)
INFO:root:DEV L2:		0.5781036(0.58)		0.81642(0.82)		0.1211013(0.12)


INFO:root:TEST Acc:		0.6282382(0.63)		0.9472131(0.95)		0.0166256(0.02)
INFO:root:TEST L2:		0.5631499(0.56)		0.8182356(0.82)		0.1124408(0.11)
INFO:root:Epoch	3/30


...

INFO:root:TEST Acc:		0.6282382(0.63)		0.9590207(0.96)		0.0166256(0.02)
INFO:root:TEST L2:		0.5631499(0.56)		0.8182356(0.82)		0.0725027(0.07)
INFO:root:Epoch	9/30


INFO:root:DEV Acc:		0.6139208(0.61)		0.9567256(0.96)		0.0206932(0.02)
INFO:root:DEV L2:		0.5781036(0.58)		0.81642(0.82)		0.0818476(0.08)


INFO:root:TEST Acc:		0.6282382(0.63)		0.9576315(0.96)		0.0166256(0.02)
INFO:root:TEST L2:		0.5631499(0.56)		0.8182356(0.82)		0.074297(0.07)
INFO:root:Epoch	10/30


INFO:root:DEV Acc:		0.6139208(0.61)		0.9563649(0.96)		0.0206932(0.02)
INFO:root:DEV L2:		0.5781036(0.58)		0.81642(0.82)		0.0792081(0.08)


INFO:root:TEST Acc:		0.6282382(0.63)		0.9565897(0.96)		0.0166256(0.02)
INFO:root:TEST L2:		0.5631499(0.56)		0.8182356(0.82)		0.0738848(0.07)


### Save Requested Model

In [40]:
torch.save(model_Requested.state_dict(), "model_PANFMR_incremental_Requested.pt")

### Load Requested Model

In [41]:
model_Requested = RequestedModel(vocabulary_size = VOCABULARY_SIZE,
                                 embedding_dim = EMBEDDING_DIM,
                                 altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                 hidden_dim = HIDDEN_DIM,
                                 requested_dim = REQUESTED_DIM,
                                 device = DEVICE)

model_Requested = model_Requested.to(DEVICE)
model_Requested.load_state_dict(torch.load("model_PANFMR_incremental_Requested.pt"))
model_Requested.eval()

RequestedModel(
  (embeddings): Embedding(1149, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (requested_classifier): Linear(in_features=100, out_features=8, bias=True)
)

In [42]:
get_scores(model_GoalPricerange,
           model_GoalArea,
           model_GoalName,
           model_GoalFood,
           model_Requested,
           model_Method,
           raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")

(0.6139208, 0.5781036, 0.9563649, 0.81642, 0.0206932, 0.0792081)

In [43]:
get_scores(model_GoalPricerange,
           model_GoalArea,
           model_GoalName,
           model_GoalFood,
           model_Requested,
           model_Method,
           raw_X_test, raw_Y_test, dataset = "dstc2_test")

(0.6282382, 0.5631499, 0.9565897, 0.8182356, 0.0166256, 0.0738848)

### Train Method Model

In [44]:
dev_goal_accuracies = []
dev_requested_accuracies = []
dev_method_accuracies = []

test_goal_accuracies = []
test_requested_accuracies = []
test_method_accuracies = []

method_early_stopping = EarlyStopping(patience = 2)

train_indices = np.arange(raw_X_train.shape[0])

for epoch in range(NUM_EPOCHS):
    
    np.random.shuffle(train_indices)
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_Method = model_Method.train()
    
    for raw_X_train_dialog, raw_Y_train_dialog in tqdm_notebook(zip(raw_X_train[train_indices], raw_Y_train[train_indices]), total = len(raw_X_train)):

        model_Method.hidden = model_Method.init_hidden()

        for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):
            
            optimizer_MethodModel.zero_grad()
        
            turn_loss = 0
            
            indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)
            
            methods = model_Method(indices, scores)
            
            for method in methods:
                
                method = method.unsqueeze(0)

                loss_method = METHOD_LOSS_FUNCTION(method,
                                                   retrieve_gold_Method(raw_Y_train_turn,
                                                                        ontology = ontology,
                                                                        device = DEVICE))
                turn_loss += loss_method
                
            turn_loss.backward(retain_graph = True)

            optimizer_MethodModel.step()
    
    dev_goal_accuracy, \
    dev_goal_l2, \
    dev_requested_accuracy, \
    dev_requested_l2, \
    dev_method_accuracy, \
    dev_method_l2 = get_scores(model_GoalPricerange,
                               model_GoalArea,
                               model_GoalName,
                               model_GoalFood,
                               model_Requested,
                               model_Method,
                               raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")
    logging.info("DEV Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_goal_accuracy,
                                                                 np.around(dev_goal_accuracy, decimals = 2),
                                                                 dev_requested_accuracy,
                                                                 np.around(dev_requested_accuracy, decimals = 2),
                                                                 dev_method_accuracy,
                                                                 np.around(dev_method_accuracy, decimals = 2)))
    logging.info("DEV L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_goal_l2,
                                                                np.around(dev_goal_l2, decimals = 2),
                                                                dev_requested_l2,
                                                                np.around(dev_requested_l2, decimals = 2),
                                                                dev_method_l2,
                                                                np.around(dev_method_l2, decimals = 2)))
    dev_goal_accuracies.append(dev_goal_accuracy)
    dev_requested_accuracies.append(dev_requested_accuracy)
    dev_method_accuracies.append(dev_method_accuracy)

    test_goal_accuracy, \
    test_goal_l2, \
    test_requested_accuracy, \
    test_requested_l2, \
    test_method_accuracy, \
    test_method_l2 = get_scores(model_GoalPricerange,
                                model_GoalArea,
                                model_GoalName,
                                model_GoalFood,
                                model_Requested,
                                model_Method,
                                raw_X_test, raw_Y_test, dataset = "dstc2_test")
    logging.info("TEST Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(test_goal_accuracy,
                                                                  np.around(test_goal_accuracy, decimals = 2),
                                                                  test_requested_accuracy,
                                                                  np.around(test_requested_accuracy, decimals = 2),
                                                                  test_method_accuracy,
                                                                  np.around(test_method_accuracy, decimals = 2)))
    logging.info("TEST L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(test_goal_l2,
                                                                 np.around(test_goal_l2, decimals = 2),
                                                                 test_requested_l2,
                                                                 np.around(test_requested_l2, decimals = 2),
                                                                 test_method_l2,
                                                                 np.around(test_method_l2, decimals = 2)))
    test_goal_accuracies.append(test_goal_accuracy)
    test_requested_accuracies.append(test_requested_accuracy)
    test_method_accuracies.append(test_method_accuracy)
        
    method_early_stopping.on_epoch_end(epoch = (epoch + 1),
                                    current_value = (dev_method_accuracy))
    
    if method_early_stopping.stop_training:
        break

plotly_plot(dev_method_accuracies, test_method_accuracies, "Method")

INFO:root:Epoch	1/30


INFO:root:DEV Acc:		0.6139208(0.61)		0.9563649(0.96)		0.8807553(0.88)
INFO:root:DEV L2:		0.5781036(0.58)		0.1958952(0.2)		0.0792081(0.08)


INFO:root:TEST Acc:		0.6282382(0.63)		0.9565897(0.96)		0.9129721(0.91)
INFO:root:TEST L2:		0.5631499(0.56)		0.1440707(0.14)		0.0738848(0.07)
INFO:root:Epoch	2/30


INFO:root:DEV Acc:		0.6139208(0.61)		0.9563649(0.96)		0.8843766(0.88)
INFO:root:DEV L2:		0.5781036(0.58)		0.1885741(0.19)		0.0792081(0.08)


INFO:root:TEST Acc:		0.6282382(0.63)		0.9565897(0.96)		0.9148194(0.91)
INFO:root:TEST L2:		0.5631499(0.56)		0.1366519(0.14)		0.0738848(0.07)
INFO:root:Epoch	3/30


...

INFO:root:TEST Acc:		0.6282382(0.63)		0.9565897(0.96)		0.9183087(0.92)
INFO:root:TEST L2:		0.5631499(0.56)		0.132724(0.13)		0.0738848(0.07)
INFO:root:Epoch	6/30


INFO:root:DEV Acc:		0.6139208(0.61)		0.9563649(0.96)		0.884118(0.88)
INFO:root:DEV L2:		0.5781036(0.58)		0.1888297(0.19)		0.0792081(0.08)


INFO:root:TEST Acc:		0.6282382(0.63)		0.9565897(0.96)		0.9082512(0.91)
INFO:root:TEST L2:		0.5631499(0.56)		0.1463414(0.15)		0.0738848(0.07)
INFO:root:Epoch	7/30


INFO:root:DEV Acc:		0.6139208(0.61)		0.9563649(0.96)		0.8766167(0.88)
INFO:root:DEV L2:		0.5781036(0.58)		0.197838(0.2)		0.0792081(0.08)


INFO:root:TEST Acc:		0.6282382(0.63)		0.9565897(0.96)		0.9069171(0.91)
INFO:root:TEST L2:		0.5631499(0.56)		0.1469578(0.15)		0.0738848(0.07)


### Save Method Model

In [45]:
torch.save(model_Method.state_dict(), "model_PANFMR_incremental_Method.pt")

### Load Method Model

In [46]:
model_Method = MethodModel(vocabulary_size = VOCABULARY_SIZE,
                           embedding_dim = EMBEDDING_DIM,
                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                           hidden_dim = HIDDEN_DIM,
                           method_dim = METHOD_DIM,
                           device = DEVICE)

model_Method = model_Method.to(DEVICE)
model_Method.load_state_dict(torch.load("model_PANFMR_incremental_Method.pt"))
model_Method.eval()

MethodModel(
  (embeddings): Embedding(1149, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (method_classifier): Linear(in_features=100, out_features=5, bias=True)
)

In [47]:
get_scores(model_GoalPricerange,
           model_GoalArea,
           model_GoalName,
           model_GoalFood,
           model_Requested,
           model_Method,
           raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")

(0.6139208, 0.5781036, 0.9563649, 0.197838, 0.8766167, 0.0792081)

In [48]:
get_scores(model_GoalPricerange,
           model_GoalArea,
           model_GoalName,
           model_GoalFood,
           model_Requested,
           model_Method,
           raw_X_test, raw_Y_test, dataset = "dstc2_test")

(0.6282382, 0.5631499, 0.9565897, 0.1469578, 0.9069171, 0.0738848)

### Plot incremental accuracies

In [83]:
def make_tracker_incremental(percentage, model_GoalPricerange, model_GoalArea, model_GoalName, model_GoalFood, model_Requested, model_Method, raw_X, raw_Y, dataset):
    
    model_GoalPricerange = model_GoalPricerange.eval()
    model_GoalArea = model_GoalArea.eval()
    model_GoalName = model_GoalName.eval()
    model_GoalFood = model_GoalFood.eval()
    model_Requested = model_Requested.eval()
    model_Method = model_Method.eval()
    
    with torch.no_grad():
        tracker_json = {}
        tracker_json["dataset"] = dataset
        tracker_json["sessions"] = []

        start_time = time.time()
        for raw_X_dialog, raw_Y_dialog in tqdm_notebook(zip(raw_X, raw_Y), total = len(raw_X)):
            
            model_GoalPricerange.hidden = model_GoalPricerange.init_hidden()
            model_GoalArea.hidden = model_GoalArea.init_hidden()
            model_GoalName.hidden = model_GoalName.init_hidden()
            model_GoalFood.hidden = model_GoalFood.init_hidden()
            model_Requested.hidden = model_Requested.init_hidden()
            model_Method.hidden = model_Method.init_hidden()
            
            session = {}
            session["session-id"] = raw_X_dialog["session-id"]
            session["turns"] = []

            for raw_X_turn, raw_Y_turn in zip(raw_X_dialog["turns"], raw_Y_dialog["turns"]):
                turn = {}
                turn["goal-labels"] = {}

                indices, scores = get_index_and_score(raw_X_turn, token_to_index, mode = "eval", device = DEVICE)
                
                incremental_index = int(np.around(percentage * len(indices))) - 1
                
                goal_foods = model_GoalFood(indices, scores)
                goal_food = goal_foods[incremental_index]
                
                goal_priceranges = model_GoalPricerange(indices, scores)
                goal_pricerange = goal_priceranges[incremental_index]
                
                goal_names = model_GoalName(indices, scores)
                goal_name = goal_names[incremental_index]
                
                goal_areas = model_GoalArea(indices, scores)
                goal_area = goal_areas[incremental_index]
                
                requesteds = model_Requested(indices, scores)
                requested = requesteds[incremental_index]
                
                methods = model_Method(indices, scores)
                method = methods[incremental_index]
                
                turn["goal-labels"]["food"] = retrieve_output_GoalFood(goal_food, ontology)
                turn["goal-labels"]["pricerange"] = retrieve_output_GoalPricerange(goal_pricerange, ontology)
                turn["goal-labels"]["name"] = retrieve_output_GoalName(goal_name, ontology)
                turn["goal-labels"]["area"] = retrieve_output_GoalArea(goal_area, ontology)
                turn["requested-slots"] = retrieve_output_Requested(requested, ontology)
                turn["method-label"] = retrieve_output_Method(method, ontology)
                
                session["turns"].append(turn)
            tracker_json["sessions"].append(session)
        end_time = time.time()
        tracker_json["wall-time"] = end_time - start_time
        
        return tracker_json

In [84]:
def get_scores_incremental(percentage, model_GoalPricerange, model_GoalArea, model_GoalName, model_GoalFood, model_Requested, model_Method, raw_X, raw_Y, dataset):
    goal_accuracy = 0
    goal_l2 = 0
    requested_accuracy = 0
    requested_l2 = 0
    method_accuracy = 0
    method_l2 = 0
    
    tracker = make_tracker_incremental(percentage, model_GoalPricerange, model_GoalArea, model_GoalName, model_GoalFood, model_Requested, model_Method, raw_X, raw_Y, dataset = dataset)
    
    with open("tracker_panfmr_incremental.json", "w") as tracker_file:
        json.dump(tracker, tracker_file)
    
    if dataset == "dstc2_train":
        !python2 dstc2/dstc2_scripts/score.py\
        --dataset dstc2_train\
        --dataroot dstc2/dstc2_traindev/data\
        --ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
        --trackfile tracker_panfmr_incremental.json\
        --scorefile tracker_panfmr_incremental.score.csv
    elif dataset == "dstc2_dev":
        !python2 dstc2/dstc2_scripts/score.py\
        --dataset dstc2_dev\
        --dataroot dstc2/dstc2_traindev/data\
        --ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
        --trackfile tracker_panfmr_incremental.json\
        --scorefile tracker_panfmr_incremental.score.csv
    else:
        !python2 dstc2/dstc2_scripts/score.py\
        --dataset dstc2_test\
        --dataroot dstc2/dstc2_test/data\
        --ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
        --trackfile tracker_panfmr_incremental.json\
        --scorefile tracker_panfmr_incremental.score.csv

    file_cat = !python2 dstc2/dstc2_scripts/report.py --scorefile tracker_panfmr_incremental.score.csv
    
    found_accuracies = False
    for line in file_cat:
        if line.startswith("Accuracy"):
            accuracies = line.split("|")
            goal_accuracy = float(accuracies[1])
            requested_accuracy = float(accuracies[2])
            method_accuracy = float(accuracies[3])
            found_accuracies = True
        if found_accuracies and line.startswith("l2"):
            l2s = line.split("|")
            goal_l2 = float(l2s[1])
            method_l2 = float(l2s[2])
            requested_l2 = float(l2s[3])
    
    return goal_accuracy, goal_l2, requested_accuracy, requested_l2, method_accuracy, method_l2

In [100]:
def plotly_plot_incremental(percentages, goal_accuracies, requested_accuracies, method_accuracies, dataset):
    plotly.offline.iplot({
                            "data": [Scatter(
                                            x = percentages,
                                            y = goal_accuracies,
                                            mode = "lines+markers",
                                            name = "{} Goal Accuracy".format(dataset),
                                            marker = dict(color = "#3498db")),
                                    Scatter(
                                            x = percentages,
                                            y = requested_accuracies,
                                            mode = "lines+markers",
                                            name = "{} Requested Accuracy".format(dataset),
                                            marker = dict(color = "#1abc9c")),
                                    Scatter(
                                            x = percentages,
                                            y = method_accuracies,
                                            mode = "lines+markers",
                                            name = "{} Method Accuracy".format(dataset),
                                            marker = dict(color = "#9b59b6"))],
                            "layout": Layout(
                                             title = "<b>Percentage - Accuracy</b>",
                                             xaxis = dict(title = "<b>Percentage</b>",
                                                          dtick = 0.1,
                                                          titlefont = dict(color = "#34495e")),
                                             yaxis = dict(title = "<b>Accuracy</b>",
                                                          titlefont = dict(color = "#34495e")),
                                             margin = Margin(b = 150))
                        })

In [87]:
percentages = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

dev_goal_accuracies = []
dev_goal_l2s = []
dev_requested_accuracies = []
dev_requested_l2s = []
dev_method_accuracies = []
dev_method_l2s = []

test_goal_accuracies = []
test_goal_l2s = []
test_requested_accuracies = []
test_requested_l2s = []
test_method_accuracies = []
test_method_l2s = []

for percentage in tqdm_notebook(percentages, total = len(percentages)):

    dev_goal_accuracy, \
    dev_goal_l2, \
    dev_requested_accuracy, \
    dev_requested_l2, \
    dev_method_accuracy, \
    dev_method_l2 = get_scores_incremental(percentage, model_GoalPricerange, model_GoalArea, model_GoalName, model_GoalFood, model_Requested, model_Method, raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")
    
    dev_goal_accuracies.append(dev_goal_accuracy)
    dev_goal_l2s.append(dev_goal_l2)
    dev_requested_accuracies.append(dev_requested_accuracy)
    dev_requested_l2s.append(dev_requested_l2)
    dev_method_accuracies.append(dev_method_accuracy)
    dev_method_l2s.append(dev_method_l2)
    
    test_goal_accuracy, \
    test_goal_l2, \
    test_requested_accuracy, \
    test_requested_l2, \
    test_method_accuracy, \
    test_method_l2 = get_scores_incremental(percentage, model_GoalPricerange, model_GoalArea, model_GoalName, model_GoalFood, model_Requested, model_Method, raw_X_test, raw_Y_test, dataset = "dstc2_test")
    
    test_goal_accuracies.append(test_goal_accuracy)
    test_goal_l2s.append(test_goal_l2)
    test_requested_accuracies.append(test_requested_accuracy)
    test_requested_l2s.append(test_requested_l2)
    test_method_accuracies.append(test_method_accuracy)
    test_method_l2s.append(test_method_l2)

In [101]:
plotly_plot_incremental(percentages, dev_goal_accuracies, dev_requested_accuracies, dev_method_accuracies, "Dev")

In [102]:
plotly_plot_incremental(percentages, test_goal_accuracies, test_requested_accuracies, test_method_accuracies, "Test")